In [17]:
import sys

sys.path.append("../")

from bunkatopics import Bunka
from bunkatopics.functions.clean_text import clean_tweet
from langchain.embeddings import HuggingFaceEmbeddings
import random
from datasets import load_dataset
import os
from dotenv import load_dotenv
from bunkatopics.functions.clean_text import clean_tweet

load_dotenv()

True

In [ ]:
dataset = load_dataset("rguo123/trump_tweets")["train"]
full_docs = dataset["content"]
full_docs = random.sample(full_docs, 2000)
full_docs = [clean_tweet(x) for x in full_docs]
full_docs = [x for x in full_docs if len(x)>50]

In [ ]:
#embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

bunka = Bunka(embedding_model=embedding_model)
bunka.fit(full_docs)

In [ ]:
from dotenv import load_dotenv

load_dotenv()

from langchain.llms import OpenAI
open_ai_generative_model = OpenAI(openai_api_key = os.getenv('OPEN_AI_KEY'))

In [ ]:
manual_axis_name = {
                    'x_left_name':'hate',
                    'x_right_name':'peace',
                   'y_top_name':'past',
                    'y_bottom_name':'future',
                    }


from bunkatopics.datamodel import BourdieuQuery
boudieu_query = BourdieuQuery(x_left_words=["this is about about men"],
                                x_right_words=["this is a about women"],
                                y_top_words=["this is about war"],
                                y_bottom_words=["this is about peace"],
                                radius_size = 0.5)


bourdieu_fig = bunka.visualize_bourdieu(
    generative_model=open_ai_generative_model,
    x_left_words=boudieu_query.x_left_words,
    x_right_words=boudieu_query.x_right_words,
    y_top_words=boudieu_query.y_top_words,
    y_bottom_words=boudieu_query.y_bottom_words,
    height=1000,
    width=1000,
    display_percent=False,
    clustering=True,
    topic_n_clusters=10,
    topic_terms=5,
    topic_top_terms_overall=500,
    topic_gen_name=True,
    convex_hull = True,
    radius_size = 0.5,
    manual_axis_name = manual_axis_name
)

#bourdieu_fig.show()

In [ ]:
bunka.start_server_bourdieu()